In [2]:
import ee
import folium 
import geehydro
import geemap
import os
import ipywidgets as widgets

In [3]:
ee.Initialize()

Initialize the Google Earth API 

In [4]:
Map = geemap.Map(center=[73.0243 , 26.2389], zoom=4)
Map.add_basemap('HYBRID') # Add Google Satellite


In [5]:
Map.draw_last_feature

## NDVI 

In [6]:
Map.draw_features

[]

# Time Slider 

Select the time frame you want  to download the NDVI images for.

In [7]:
start_year = widgets.IntSlider(description='Start Year:', value=2000, min=2000, max=2020)
end_year = widgets.IntSlider(description='End Year:', value=2020, min=2000, max=2020)
start_month = widgets.IntSlider(description='Start Month:', value=5, min=1, max=12)
end_month = widgets.IntSlider(description='End Month:', value=10, min=1, max=12)
hbox3 = widgets.HBox([start_year, end_year, start_month, end_month])
hbox3


Time slider widget code 

In [8]:
#start month and year
sy = str(start_year.value)
sy
#start_month.value

sm = str(start_month.value)
if start_month.value < 10 :
    sm = '0'+sm


In [8]:
#end month and year
ey = str(end_year.value)
em = str(end_month.value)
if end_month.value < 10 :
    em = '0'+em


In [9]:
startday = sy+'-'+sm+'-01'
endday = ey+'-'+em+'-28'



In [10]:
print("Your selected start date " + startday)
print("Your selected end date " + endday)


Your selected start date 2000-05-01
Your selected end date 2020-10-28


# ImageCollection to Image


In [11]:
dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_8DAY_NDVI').filterDate(startday, endday)

colorized = dataset.select('NDVI')
vis_params = {'palette' : ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b' , '#a6d96a', '#66bd63', '#1a9850' ] }

#taking median
image = colorized.reduce(ee.Reducer.median());


In [12]:
Map.setCenter(6.746, 46.529, 6)
Map.addLayer(image, vis_params, 'Colorized')

# Interactive Map



In [13]:
import ipywidgets as widgets
from ipyleaflet import WidgetControl
from geemap import geojson_to_ee

colorizedVis = {'palette' : ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5'] }

Map.setCenter(6.746, 46.529, 6)
Map.addLayer(colorized, colorizedVis, 'Colorized')


# Get basemap layers
base_layers = Map.layers


# An empty list for storing drawing geometries
feat_list = []

# Get the DrawControl
dc = Map.draw_control

# Handle draw events
def handle_draw(self, action, geo_json):

    geom = geojson_to_ee(geo_json, False)
    feature = ee.Feature(geom)
    feat_list.append(feature)
    collection = ee.FeatureCollection(feat_list)
    clip_image = image.clipToCollection(collection)
    
    Map.layers = base_layers[:3]
    Map.addLayer(clip_image, vis_params, 'SRTM DEM')
#     Map.addLayer(ee.Image().paint(collection, 0, 2), {'palette': 'red'}, 'EE Geometry')
    Map.addLayer(collection, {}, 'Drawing Features')

dc.on_draw(handle_draw)

# Add a button to the map
button = widgets.Button(description="Clear drawings")
btn_control = WidgetControl(widget=button, position='bottomright')
Map.add_control(btn_control)

# Handle click event
def on_button_clicked(b):
    dc.clear()

button.on_click(on_button_clicked)

Map


Map(center=[46.529, 6.746], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(v…

In [14]:
dc.last_draw


{'type': 'Feature',
 'properties': {'style': {'stroke': True,
   'color': '#3388ff',
   'weight': 4,
   'opacity': 0.5,
   'fill': True,
   'fillColor': None,
   'fillOpacity': 0.2,
   'clickable': True}},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[73.151993, 19.080797],
    [73.151993, 19.12654],
    [73.217908, 19.12654],
    [73.217908, 19.080797],
    [73.151993, 19.080797]]]}}

In [18]:
#dc.draw_last_feature

# Export as tif selected region 

In [15]:
feature = Map.draw_last_feature
#feature = Map.draw_features

if feature is None:
    geom = ee.Geometry.Polygon([[[-115.413031, 35.889467],
    [-115.413031, 36.543157],
    [-114.034328, 36.543157],
    [-114.034328, 35.889467],
    [-115.413031, 35.889467]]])
    feature = ee.Feature(geom, {})

roi = feature.geometry()




# Saving the extracted coordinates as Tiff on local machine


In [16]:
#saving on system

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads/TIFFd')
filename = os.path.join(out_dir, 'landsat'+startday+'.tif')

image.clip(roi).unmask()
#geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=False)
geemap.ee_export_image(image, filename=filename, scale=90, region=roi, file_per_band=True)


Generating URL ...
Please wait ...
Data downloaded to /Users/rishwari/Downloads/TIFFd
